In [8]:
import os
import sys

In [9]:
# Detect if running in Google Colab environment.
# If so, then clone/install SmartStockRL package from GitHub.
# Otherwise, use locally.
try:
    import google.colab
    print('IN COLAB')

    # Clone repo.
    repo = "smart-stock-rl"
    repo_url = f"https://github.com/zanderman/{repo}.git"
    repo_path = f"/content/{repo}"
    !rm -rf $repo_path
    !git clone --branch develop $repo_url

    # Install repo to ensure dependencies are resolved.
    !pip install --upgrade $repo_path

    # Add package location to path.
    sys.path.insert(0, repo_path)

    # Set dataset root path.
    dataset_root = '~/dataset'

except:
    dataset_root = '~/Desktop'
    print('NOT IN COLAB')

# Import package.
import smart_stock as ss

NOT IN COLAB


In [10]:
# Import other necessary packages after we know that imports are available.
import gym
import matplotlib.pyplot as plt
import numpy as np
import torch

In [11]:
# Request user for Kaggle login if JSON file does not exist.
if not os.path.exists(os.path.expanduser("~/.kaggle/kaggle.json")):
    import getpass
    username = getpass.getpass(prompt="Kaggle Username: ")
    api_key = getpass.getpass(prompt="Kaggle API KEY: ")
    os.environ["KAGGLE_USERNAME"] = username
    os.environ["KAGGLE_KEY"] = api_key

In [12]:
# Install future annotations for <3.7
if sys.version_info < (3,7):
    !pip install future-annotations

In [13]:
# Prepare dataset.
stocks = ['aapl', 'nvda', 'dis', 'ko', 'pg']
path = os.path.expanduser(dataset_root)
dataset = ss.datasets.HugeStockMarketDataset(path, quiet=False)

TypeError: __init__() got an unexpected keyword argument 'quiet'

In [ ]:
df = dataset['aapl']
print(df.info)

<bound method DataFrame.info of             Date       Open       High        Low      Close    Volume  \
0     1984-09-07    0.42388    0.42902    0.41874    0.42388  23220030   
1     1984-09-10    0.42388    0.42516    0.41366    0.42134  18022532   
2     1984-09-11    0.42516    0.43668    0.42516    0.42902  42498199   
3     1984-09-12    0.42902    0.43157    0.41618    0.41618  37125801   
4     1984-09-13    0.43927    0.44052    0.43927    0.43927  57822062   
...          ...        ...        ...        ...        ...       ...   
8359  2017-11-06  171.75000  174.36000  171.10000  173.63000  34901241   
8360  2017-11-07  173.29000  174.51000  173.29000  174.18000  24424877   
8361  2017-11-08  174.03000  175.61000  173.71000  175.61000  24451166   
8362  2017-11-09  174.48000  175.46000  172.52000  175.25000  29533086   
8363  2017-11-10  175.11000  175.38000  174.27000  174.67000  25130494   

      OpenInt  
0           0  
1           0  
2           0  
3           0  